In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/dataset_dict_prune'

# 01 - Generate the Correlation Maps for Training

Enter the path to the dictionary generated by Notebook 01

In [2]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_01/redo2/conn/tmp/dataset_dict_prune.json'

What correlation method would you like to use?
- Pearson is very fast, but depends on linearity and is sensitive to outliers
- Spearman is slower, but much more robust

In [3]:
correlation = 'spearman'

Option A - Generate R-maps to optimize 

In [4]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
from calvin_utils.ccm_utils.stat_utils import CorrelationCalculator
data_loader = DataLoader(dict_path)
correlation_calculator = CorrelationCalculator(method=correlation, verbose=False)
corr_map_dict = correlation_calculator.generate_correlation_maps(data_loader)

Evaluating ANT DBS SANTE
Evaluating STN DBS
Evaluating ANT DBS Maastricht
Evaluating Epilepsy iEEG
Evaluating Alzheimer Disease
Evaluating Acute Stroke
Evaluating Multiple Sclerosis
Evaluating Penetrating TBI
Evaluating Parietal TMS Hebscher
Evaluating Parietal TMS Wang
Evaluating Parietal TMS Hermiller 2019a
Evaluating Parietal TMS Hermiller 2019b


Option B - Import a-priori maps for optimization

In [5]:
# nifti_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/published_networks/Canonical Networks (Yeo 2011)/*'
# mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [6]:
# from calvin_utils.ccm_utils.nifti_utils import DatasetNiftiImporter
# corr_map_dict = DatasetNiftiImporter.import_niftis_to_dict(mask_path, nifti_path)
# corr_map_dict

# 02 - Get the Data for Testing

In [7]:
testing_dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_01/redo2/conn/tmp/dataset_dict_prune.json'

In [8]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
test_data_loader = DataLoader(testing_dict_path)

# 03 - Run Optimizer

Generate the Convergent Maps and Save the Individual Correlation Maps

In [9]:
from calvin_utils.ccm_utils.optimization.convergent_map_optimizer import NiftiOptimizer
optimizer = NiftiOptimizer(corr_map_dict, test_data_loader, load_in_time=False)
optimized_map, blended_map = optimizer.optimize(store_iters=True, store_best=True)

===Imaging Optimizer Initialized===
Initializing: 
	 Weights: (1, 12)  
	 Training Maps: (12, 228483)
===        Training Data        ===
	 Training Dataset (For weighted average): ANT DBS SANTE
	 Training Dataset (For weighted average): STN DBS
	 Training Dataset (For weighted average): ANT DBS Maastricht
	 Training Dataset (For weighted average): Epilepsy iEEG
	 Training Dataset (For weighted average): Alzheimer Disease
	 Training Dataset (For weighted average): Acute Stroke
	 Training Dataset (For weighted average): Multiple Sclerosis
	 Training Dataset (For weighted average): Penetrating TBI
	 Training Dataset (For weighted average): Parietal TMS Hebscher
	 Training Dataset (For weighted average): Parietal TMS Wang
	 Training Dataset (For weighted average): Parietal TMS Hermiller 2019a
	 Training Dataset (For weighted average): Parietal TMS Hermiller 2019b
===         Testing Data        ===
	 Testing Dataset (To predict outcomes using weighted average): ANT DBS SANTE
	 Testing Dat

Optimizing Weights, Best RMS Rho = -inf:  34%|███▍      | 170/500 [23:21<45:21,  8.25s/it] 

[Convergence] Loss change 8.288e-07 < loss_tol.
Best weights: [[-0.10824107  0.02319545 -0.08322448 -0.10984927 -0.0509831  -0.09902986
  -0.09712327 -0.08385605 -0.0698106  -0.09316267 -0.07051128 -0.11101288]]


Blend optimize: best_alpha=0.2964824120603015, best_W=[[0.02653489 0.06550351 0.03395187 0.02605809 0.04351087 0.02926585
  0.02983112 0.03376462 0.03792885 0.03100537 0.03772111 0.0257131 ]]


Save Best Map
- Must set store_best=True

In [10]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average_flipped_lesions/convergence_maps/optimized'

In [17]:
from calvin_utils.ccm_utils.ccm_correlation_analysis import save_individual_r_maps
save_individual_r_maps(data=optimized_map, out_dir=out_dir, filename='optimized_convergent_map_best.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save Blended Map

In [ ]:
from calvin_utils.ccm_utils.ccm_correlation_analysis import save_individual_r_maps
save_individual_r_maps(data=blended_map, out_dir=out_dir, filename='optimized_convergent_map_blended.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save All Intermediate Maps
- Must have set store_iters=True

In [19]:
from calvin_utils.ccm_utils.ccm_correlation_analysis import save_individual_r_maps
for i, vol in enumerate(optimizer.iter_maps):
    save_individual_r_maps(data=vol, 
                           out_dir=out_dir, 
                           filename=f"iteration_maps/iter_{i:03d}.nii.gz", 
                           mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
                           verbose=False)

Thats all there is to it

-Calvin